In [7]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn

In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [9]:
NUM_CHANNELS = 3
LEARNING_RATE = 1e-3
EPOCHS = 25
BATCH_SIZE = 8
IMAGE_HEIGHT = 1080
IMAGE_WIDTH = 1920

In [10]:
data_filepath = "./final_harvest_data.csv"
data = pd.read_csv(data_filepath)

In [ ]:
def dataloader(filepath):
    

In [11]:
class conv_block(nn.Module):
    def __init__(self, input_channels, output_channels):
        super().__init__()
        self.conv1 = nn.Conv2d(input_channels, output_channels, kernel_size = 3, stride =1, padding=1)
        self.bn1 = nn.BatchNorm2d(output_channels)
        self.conv2 = nn.Conv2d(output_channels, output_channels, kernel_size = 3, stride = 1, padding=1)
        self.bn2 = nn.BatchNorm2d(output_channels)
        self.relu = nn.ReLU()

        def forward(self, input):
            x = self.conv1(input)
            x = self.bn1(x)
            x = self.conv2(x)
            x = self.bn2(x)
            x = self.relu(x)
            return x

In [12]:
class encoder_block(nn.Module):
    def __init__(self, input_channels, output_channels):
        super().__init__()
        self.conv = conv_block(input_channels, output_channels)
        self.pool = nn.MaxPool2d((2,2))

    def forward(self, inputs):
        x = self.conv(inputs)
        p = self.pool(x)
        return x, p

In [13]:
class decoder_block(nn.Module):
    def __init__(self, input_channels, output_channels):
        super().__init__()
        self.up = nn.ConvTranspose2d(input_channels, output_channels, kernel_size=3, stride=1, padding=1)
        self.conv = conv_block(output_channels*2, output_channels)
        
    def forward(self, inputs, skip):
        x = self.up(inputs)
        x = torch.cat([x, skip], axis=1)
        x = self.conv(x)
        return x

In [14]:
class built_unet(nn.Module):
    def __init__(self):
        super().__init__()

        self.e1 = encoder_block(3, 64)
        self.e2 = encoder_block(64, 128)
        self.e3 = encoder_block(128, 256)
        self.e4 = encoder_block(256, 512)

        self.b = conv_block(512,1024)

        self.d1 = decoder_block(1024, 512)
        self.d2 = decoder_block(512, 256)
        self.d3 = decoder_block(256, 128)
        self.d4 = decoder_block(128, 64)

        self.outputs = nn.Conv2d(64, 1, kernel_size=1, padding=0)

    def forward(self, inputs):

        s1, p1 = self.e1(inputs)
        s2, p2 = self.e2(p1)
        s3, p3 = self.e3(p2)
        s4, p4 = self.e4(p3)

        b = self.b(p4)

        d1 = self.d1(b, s4)
        d2 = self.d2(d1, s3)
        d3 = self.d3(d2, s2)
        d4 = self.d4(d3, s1)

        outputs = self.outputs(d4)

        return outputs
        
        